In [3]:
import msprime
import numpy as np

In [13]:
RHO_HUMAN = 1.6*10e-9
MU_HUMAN = 1.25*10e-8
LENGTH_NORMALIZE_CONST = 4
L = int(3e9)
data = msprime.simulate(sample_size=100,
                        recombination_rate=RHO_HUMAN,
                        mutation_rate=MU_HUMAN,
                        random_seed=42,
                        length=L
                       )

In [14]:
haplotype = np.zeros(shape=L, dtype=int)
mutations = [min(L-1, round(mutation.position)) for mutation in data.mutations()]

for mutation in mutations: haplotype[mutations] = 1

del mutations

In [30]:
recombination_points = []
coal_times = []
min_t = 10**100
max_t = 0

for tree in data.trees():
    point = round(tree.get_interval()[0])
    if point not in recombination_points:
        recombination_points.append(point)
        t = tree.total_branch_length/LENGTH_NORMALIZE_CONST
        coal_times.append(t)
        min_t = min(t, min_t)
        max_t = max(t, max_t)
        
recombination_points.append(L)

In [22]:
N = 20
a = (-np.log(max_t) + N*np.log(min_t))/(N-1)
B = (-np.log(min_t) + np.log(max_t))/(N-1) + 10**(-10)

def to_T(time):
    return round((np.log(time)-a)/B)

step_of_discratization = max_t/N

def discretization(t):
    return min(int(t/step_of_discratization) + 1, N)

d_times = [to_T(t) for t in coal_times]


In [19]:
len(recombination_points)

924

In [41]:
prioty_distribution = [0.0 for i in range(N+1)]
for i in range(1,len(recombination_points)):
    l = recombination_points[i] - recombination_points[i-1]
    prioty_distribution[d_times[i-1]] += l
    
prioty_distribution = [p/L
                       for p in prioty_distribution]

In [42]:
prioty_distribution

[0.0,
 0.0031179913333333332,
 0.022126026333333333,
 0.06122505166666667,
 0.066563773,
 0.09347601066666666,
 0.080505526,
 0.11620970833333333,
 0.07653717133333333,
 0.078465814,
 0.08699415766666667,
 0.08240898266666667,
 0.13189825733333332,
 0.070251681,
 0.020561687666666665,
 0.006985862666666667,
 0.002389426,
 4.607e-05,
 0.0,
 0.0,
 0.00023680233333333333]

In [43]:
intervals_starts = [np.e**(B*i+a) for i in range(N)]

In [44]:
intervals_starts

[3.1057307005812773,
 3.3109884011895345,
 3.529811580495318,
 3.763096780804947,
 4.011799796893816,
 4.276939592001285,
 4.559602472633626,
 4.860946539279629,
 5.182206431273955,
 5.524698385248436,
 5.889825627896359,
 6.279084125144471,
 6.6940687112876756,
 7.136479623198172,
 7.608129466380382,
 8.11095064141236,
 8.647003261200602,
 9.218483591486144,
 9.827733049183573,
 10.47724779542015]